In [28]:
#grabs all files in folder with particular file type

import glob
import os
print(os.getcwd() + "\n")

dbc = 'ms' #drainage basin code, others are co, ct, ms
x = glob.glob(f'clipped__{dbc}/*.nc')
print(len(x))

/work/albertl_uri_edu/fluxtoflow/nldas

2617


In [10]:
import numpy as np

In [29]:
## all file size stuff, showing you what numpy can do.

y = np.asarray([os.path.getsize(i)*1e-6 for i in x]) #megabytes found using list comprehension
print(y[0]) # prints first file's size
print(y.shape) # shape of file list

print(f'y mean (mb): {np.mean(y)} '
      f'max: {np.max(y)} '
      f'min: {np.min(y)} '
      f'y sum: {np.sum(y)} '
     )
#strange that the numbers cut off differently.

0.402225
(2617,)
y mean (mb): 0.40222499999999994 max: 0.402225 min: 0.402225 y sum: 1052.622825 


In [30]:
import xarray as xr

# we're not using grib files, so disregard the warning here.

In [35]:
import torch
import arrow
import time

def nc2torch(i):
    return torch.from_numpy(np.asarray(i)).unsqueeze(0)


def combine(dbc):
    print(arrow.now('US/Eastern'))
    t0 = time.time()
    x = glob.glob(f'clipped__{dbc}/*.nc')
    xx = xr.open_dataset(x[0]) # just as a reference for shape in dset...
    dset = torch.empty([0,2,xx.Qs_summed.shape[0],xx.Qs_summed.shape[1]]) # instantiates empty torch tensor.

    #grabs every daily file in folder, opens it, grabs surface and subsurface flow, and combines them into a pytorch tensor for the neural network.
    for i in sorted(x):
        j = xr.open_dataset(i)
        z0 = nc2torch(j.Qs_summed) # calling nc2torch function, which is taking the numpy matrix and turning it into tensor.
        z1 = nc2torch(j.Qsb_summed)
        z2 = torch.cat((z0,z1),axis=0).unsqueeze(0)
        dset = torch.cat((dset,z2),axis=0)
    t1 = time.time()
    print(f'{dbc}\t{dset.shape}\t{t1-t0} seconds')
    print(arrow.now('US/Eastern'))
    torch.save(dset,f'nldas_{dbc}.pt')
    return _

x = ['br','co','ct','ms']
for i in x:
    combine(i)

2022-06-21T13:20:40.962765-04:00
br	torch.Size([2617, 2, 16, 15])	95.26270651817322 seconds
2022-06-21T13:22:16.227419-04:00
2022-06-21T13:22:16.309504-04:00
co	torch.Size([2617, 2, 100, 91])	275.13034653663635 seconds
2022-06-21T13:26:51.441420-04:00
2022-06-21T13:26:53.489200-04:00
ct	torch.Size([2617, 2, 32, 16])	97.14105033874512 seconds
2022-06-21T13:28:30.632309-04:00
2022-06-21T13:28:30.774961-04:00
ms	torch.Size([2617, 2, 167, 288])	948.301483631134 seconds
2022-06-21T13:44:19.078013-04:00


In [37]:
torch.save(dset,'nldas_ms.pt') # i like to have this in a different field so i can execute it as i please.